In [11]:
#Load libraries
import numpy as np
from sklearn.decomposition import PCA
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
#Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
import sklearn.metrics
from sklearn.model_selection import GridSearchCV
from sklearn import decomposition
from sklearn.model_selection import StratifiedKFold

## First Analaysis with CNN features and PCA (28.04)

In [3]:
import pickle
with open(r"training_set.p", "rb") as input_file:
    data_full = pickle.load(input_file)

In [5]:
# Input data here
y_full = data_full[1]
x_full = data_full[0]
#Label and x-data
#y_full =  np.random.binomial(1, 0.5, size=600)
#x_full = np.random.rand(600,750)
pd.DataFrame(y_full).describe()
#pd.DataFrame(x_full).describe()

,0
count,863.000000
mean,0.492468
std,0.500233
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [6]:
#Try PCA first
pca_all = decomposition.PCA(n_components = 100) #use 150 if you want more variance explained
pca_all.fit(x_full)


PCA(n_components=100)

In [7]:
#get new X
x_full = pca_all.transform(x_full)

In [8]:
pd.DataFrame(x_full).describe()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,863.000000,863.000000,8.630000e+02,863.000000,8.630000e+02,8.630000e+02,863.000000,8.630000e+02,8.630000e+02,8.630000e+02,...,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02
mean,0.000004,-0.000005,-5.967371e-08,-0.000003,-4.033501e-07,9.509117e-07,-0.000001,4.337395e-07,-4.602611e-07,6.809986e-07,...,4.748342e-07,3.320731e-07,9.612025e-07,-4.800143e-07,-5.214543e-08,1.187949e-08,7.327987e-08,2.462922e-07,6.967113e-07,-1.439697e-07
std,2.903462,1.271854,1.227888e+00,1.146521,9.209765e-01,8.445962e-01,0.762597,6.650893e-01,6.480259e-01,5.509889e-01,...,8.133917e-02,7.973859e-02,7.949471e-02,7.835900e-02,7.813580e-02,7.709689e-02,7.572611e-02,7.426398e-02,7.371980e-02,7.308110e-02
min,-5.924026,-3.423473,-4.489543e+00,-2.888272,-3.092029e+00,-3.133666e+00,-2.651993,-2.150131e+00,-1.800066e+00,-1.475130e+00,...,-2.998728e-01,-2.525188e-01,-3.341567e-01,-2.844992e-01,-2.406101e-01,-3.010074e-01,-2.647325e-01,-2.987284e-01,-2.613409e-01,-3.033389e-01
25%,-2.247991,-0.767915,-8.127704e-01,-0.778933,-6.250591e-01,-5.001017e-01,-0.469975,-4.550265e-01,-4.141251e-01,-3.813923e-01,...,-5.125899e-02,-5.083697e-02,-4.484938e-02,-4.553493e-02,-4.907078e-02,-4.968786e-02,-4.801110e-02,-4.491391e-02,-4.337645e-02,-4.811698e-02
50%,-0.179334,-0.083919,8.522185e-02,-0.130937,-3.642917e-02,1.841158e-02,-0.042228,-5.638554e-02,-3.280040e-03,-3.306641e-02,...,1.066649e-03,1.551168e-03,-2.393065e-03,-2.238478e-03,2.818268e-03,-1.974091e-03,-2.960294e-03,5.777217e-04,1.859106e-03,1.152530e-03
75%,2.028248,0.671143,8.434451e-01,0.634593,6.103609e-01,5.436082e-01,0.455044,3.840627e-01,3.983361e-01,3.010277e-01,...,5.111381e-02,4.985153e-02,4.967995e-02,4.654672e-02,4.797407e-02,4.760538e-02,4.534797e-02,4.254235e-02,4.096385e-02,4.419227e-02
max,9.400922,5.528559,4.758941e+00,5.382068,3.156897e+00,3.347646e+00,3.713265,2.702808e+00,3.812312e+00,2.124125e+00,...,4.451458e-01,5.022255e-01,3.464682e-01,3.098977e-01,3.045732e-01,3.354615e-01,3.053100e-01,3.092036e-01,3.295572e-01,3.620229e-01


In [12]:
#Try Random forest and other classifiers
rf_split = RandomForestClassifier(random_state=1337,criterion="entropy",max_depth=10, min_samples_split=15,n_estimators=1000) 
#rf_split.fit(x_full,y_full)
#Support Vector Classifier, C should actually be seperately tuned 
svm_split = sklearn.svm.SVC(random_state=1337,C=0.5) 
#Ridge Classifier, alpha should actually be seperately tuned
ridge_split = sklearn.linear_model.RidgeClassifier(random_state=1337,normalize=True,alpha=2)
#Ada Boost 
ada_split = sklearn.ensemble.AdaBoostClassifier(n_estimators=1000,random_state=1337)

In [13]:
scores_rf=cross_val_score(rf_split, x_full, y_full, n_jobs=-1,scoring=make_scorer(accuracy_score),  cv = StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579))
scores_rf

array([0.64367816, 0.77011494, 0.66666667, 0.51162791, 0.62790698,
       0.6744186 , 0.62790698, 0.77906977, 0.63953488, 0.58139535])

In [14]:
scores_svm=cross_val_score(svm_split, x_full, y_full, cv= StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_svm

array([0.63218391, 0.73563218, 0.63218391, 0.56976744, 0.62790698,
       0.60465116, 0.60465116, 0.80232558, 0.58139535, 0.58139535])

In [15]:
scores_ridge=cross_val_score(ridge_split, x_full, y_full, cv=StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_ridge

array([0.65517241, 0.77011494, 0.71264368, 0.55813953, 0.65116279,
       0.69767442, 0.62790698, 0.73255814, 0.65116279, 0.60465116])

In [27]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 166.7 MB 782 kB/s eta 0:00:01
  distutils: /home/haicore-project-covchl/yu4305/.local/lib/python3.6/site-packages
  sysconfig: /home/haicore-project-covchl/yu4305/.local/lib64/python3.6/site-packages
  user = True
  home = None
  root = None
  prefix = None
distutils: /home/haicore-project-covchl/yu4305/.local/lib/python3.6/site-packages
sysconfig: /home/haicore-project-covchl/yu4305/.local/lib64/python3.6/site-packages
user = True
home = None
root = None
prefix = None


In [16]:
scores_ada=cross_val_score(ada_split, x_full, y_full, cv=StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_ada

array([0.5862069 , 0.57471264, 0.57471264, 0.44186047, 0.62790698,
       0.59302326, 0.60465116, 0.6627907 , 0.53488372, 0.60465116])

In [30]:
#Now use XGBoost
import sklearn
import xgboost as xgb
xgb_split = xgb.XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=10, min_child_weight=2, objective="binary:logistic", subsample=0.9, colsample_bytree=0.8, seed=1337)


In [31]:
scores_xgb=cross_val_score(xgb_split, x_full, y_full, cv=StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_xgb

array([0.65517241, 0.72413793, 0.67816092, 0.45348837, 0.58139535,
       0.69767442, 0.6627907 , 0.68604651, 0.62790698, 0.60465116])

In [32]:
print("Mean 10-fold CV Accuracy: ")
print("Random Forest: ",np.mean(scores_rf))
print("Support Vector Machine: ", np.mean(scores_svm))
print("Ridge Classifier: ",np.mean(scores_ridge))
print("Ada Boost: ", np.mean(scores_ada))
print("XGB Boost: ", np.mean(scores_xgb))

Mean 10-fold CV Accuracy: 
Random Forest:  0.6522320235231223
Support Vector Machine:  0.6372093023255815
Ridge Classifier:  0.6661186848436248
Ada Boost:  0.5805399625768511
XGB Boost:  0.6371424752739909


## Now look at Imputed Clinical Data and repeat the analysis

In [65]:
#####################
#Look at imputed clinical data
#import data

clinical_knn = pd.read_csv("trainSet_knn.txt")
clinical_imp = pd.read_csv("trainSet_imputed.txt")

clinical_knn.describe()
x_knn = clinical_knn.drop('Prognosis',1)
y_knn = clinical_knn[['Prognosis']]

x_imp = clinical_imp.drop('Prognosis',1)
y_imp = clinical_imp[['Prognosis']]
print(x_knn)
#y_knn = clinical_knn["Prognosis"]

     Unnamed: 0  Hospital        Age  Sex  Temp_C  Cough  \
0             0       2.0  35.913889  0.0   39.30    1.0   
1             1       2.0  57.266667  0.0   37.00    0.0   
2             2       2.0  79.263889  0.0   37.80    1.0   
3             3       2.0  82.000000  0.0   38.00    1.0   
4             4       2.0  60.791667  1.0   37.00    1.0   
..          ...       ...        ...  ...     ...    ...   
858         858       4.0  51.000000  0.0   37.51    0.0   
859         859       4.0  57.000000  0.0   37.40    1.0   
860         860       4.0  38.000000  0.0   37.14    0.0   
861         861       4.0  92.000000  1.0   38.00    0.0   
862         862       4.0  80.000000  0.0   38.40    0.0   

     DifficultyInBreathing    WBC     CRP  Fibrinogen    LDH  Ddimer  \
0                      0.0   5.76   43.40       651.0  387.0   157.0   
1                      0.0  11.48   64.00       700.0  338.0   601.0   
2                      0.0   6.21  115.30       698.0  356.0   

In [57]:
#Try Random forest and other classifiers
def ml_models ():
    rf_clinic = RandomForestClassifier(random_state=1337,criterion="entropy",max_depth=10, min_samples_split=15,n_estimators=1000) 
    #rf_split.fit(x_full,y_full)
    #Support Vector Classifier, C should actually be seperately tuned 
    svm_clinic = sklearn.svm.SVC(random_state=1337,C=0.5) 
    #Ridge Classifier, alpha should actually be seperately tuned
    ridge_clinic = sklearn.linear_model.RidgeClassifier(random_state=1337,normalize=True,alpha=2)
    #Ada Boost 
    ada_clinic = sklearn.ensemble.AdaBoostClassifier(n_estimators=1000,random_state=1337)
    #XGB Boost
    #xgb_clinic = xgb.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=5, min_child_weight=2, objective="binary:logistic", subsample=0.9, colsample_bytree=0.8, seed=1337)
    return rf_clinic , svm_clinic, ridge_clinic, ada_clinic#, xgb_clinic

In [58]:
def ret_cv_scores(model,x_full,y_full):
    scores=cross_val_score(model, x_full, y_full, cv=StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
    return scores

In [61]:
rf_clinic_knn , svm_clinic_knn, ridge_clinic_knn, ada_clinic_knn = ml_models() #,xgb_clinic_knn

In [62]:
for i in [rf_clinic_knn , svm_clinic_knn, ridge_clinic_knn, ada_clinic_knn]:#, xgb_clinic_knn]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = x_knn, y_full = y_knn)
    print(np.mean(score_temp))
    

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.7579256883186314
SVC(C=0.5, random_state=1337)
0.634923817161187
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.7416733493718258
AdaBoostClassifier(n_estimators=1000, random_state=1337)
0.6952686447473939


In [63]:
rf_clinic_imp , svm_clinic_imp, ridge_clinic_imp, ada_clinic_imp = ml_models() #, xgb_clinic_imp

In [66]:
for i in [rf_clinic_imp , svm_clinic_imp, ridge_clinic_imp, ada_clinic_imp]:#, xgb_clinic_imp]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = x_imp, y_full = y_imp)
    print(np.mean(score_temp))

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.745175086875167
SVC(C=0.5, random_state=1337)
0.5376770916867148
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.7080459770114943
AdaBoostClassifier(n_estimators=1000, random_state=1337)
0.6906041165463781


## Use Data from Autoencoder (not yet implemented)

In [181]:
#Try Autoencoded Data

clinical_y = pd.read_csv("Data Auto Encoder/trainSet.txt")
clinical_auto = pd.read_csv("Data Auto Encoder/simon_TRAIN_impute_classify.csv")

clinical_auto.describe()
x_auto = clinical_auto.drop(['PatientID','ImageFile','Hospital','Prognosis'],1)
y_auto = clinical_y[['Prognosis']]


In [182]:
x_auto

,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,Fibrinogen,LDH,Ddimer,Ox_percentage,PaO2,SaO2,pH,CardiovascularDisease,RespiratoryFailure
0,35.913889,0.0,39.300000,1.0,0.0,5.76,43.40,651.000000,387.000000,157.000000,94.000000,71.863914,95.122742,7.651553,0.0,0.0
1,57.266667,0.0,37.000000,0.0,0.0,11.48,64.00,700.000000,338.000000,601.000000,94.000000,75.000000,96.900000,7.420000,0.0,0.0
2,79.263889,0.0,37.800000,1.0,0.0,6.21,115.30,698.000000,356.000000,448.000000,94.000000,63.000000,94.600000,7.390000,1.0,0.0
3,82.000000,0.0,38.000000,1.0,0.0,7.28,149.30,513.000000,482.000000,959.028015,97.000000,68.000000,96.300000,7.460000,0.0,0.0
4,60.791667,1.0,37.000000,1.0,0.0,6.37,20.70,577.426819,260.697968,210.000000,93.000000,74.015709,97.300000,7.639133,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,51.000000,0.0,38.222656,0.0,1.0,14.30,22.79,550.000000,368.000000,5027.000000,92.603195,70.067528,92.252815,7.590069,0.0,0.0
859,57.000000,0.0,38.613785,1.0,0.0,5.10,8.93,757.000000,451.000000,933.251465,95.931114,72.046349,95.225761,7.650484,0.0,0.0
860,38.000000,0.0,38.240093,0.0,1.0,7.30,0.23,611.945129,386.043854,1073.000000,92.770645,70.120804,92.430649,7.593248,0.0,0.0
861,92.000000,1.0,38.000000,0.0,1.0,13.50,3.77,533.000000,358.000000,2154.000000,93.311035,75.700000,93.616997,7.360000,0.0,0.0


In [183]:
rf_auto, svm_auto, ridge_auto, ada_auto = ml_models() #,xgb_clinic_knn

In [184]:
for i in [rf_auto, svm_auto, ridge_auto, ada_auto]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = x_auto, y_full = y_auto)
    print(np.mean(score_temp))

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.7625233894680565
SVC(C=0.5, random_state=1337)
0.6594627105052125
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.7312483293237102
AdaBoostClassifier(n_estimators=1000, random_state=1337)
0.6848703555199145


In [151]:
#read in combined data
combined_imp = pd.read_pickle("df_full.p")
cnn_imp = pd.read_pickle("Xy_imgs.p")
#print(combined_imp.describe())
#print(combined_imp.shape)
x_combined = combined_imp.drop(['Prognosis','PatientID','ImageFile','Hospital'],1)

In [ ]:
y_combined = combined_imp["Prognosis"].apply(lambda x: 1 if x == "SEVERE" else 0)

In [153]:
cnn_fin = cnn_imp[0]

In [155]:
cnn_fin

array([[0.        , 0.        , 0.        , ..., 0.        , 0.05012614,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.03571007,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00273884, 0.02464284,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.13519804,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.24280955,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00892852,
        0.        ]], dtype=float32)

In [156]:
pca_cnn = decomposition.PCA(n_components = 100) #use 150 if you want more variance explained
pca_cnn.fit(cnn_fin)

PCA(n_components=100)

In [165]:
x_new = pca_all.transform(cnn_fin)
y_comb = cnn_imp[1]

In [172]:
x_comb = pd.concat([x_knn,pd.DataFrame(x_new)],axis=1)


In [162]:
rf_comb, svm_comb, ridge_comb, ada_comb = ml_models()

In [174]:
for i in [rf_comb, svm_comb, ridge_comb, ada_comb]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = x_comb, y_full = y_comb)
    print(np.mean(score_temp))

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.6452419139267576
SVC(C=0.5, random_state=1337)
0.5063753007217322
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.6487703822507351
AdaBoostClassifier(n_estimators=1000, random_state=1337)
0.5897353648757017


## Use CNN only

In [175]:
rf_cnn, svm_cnn, ridge_cnn, ada_cnn = ml_models()

In [177]:
for i in [rf_cnn, svm_cnn, ridge_cnn, ada_cnn]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = pd.DataFrame(x_new), y_full = y_comb)
    print(np.mean(score_temp))

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.642956428762363
SVC(C=0.5, random_state=1337)
0.625554664528201
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.6464314354450682
AdaBoostClassifier(n_estimators=1000, random_state=1337)
0.6048115477145148


## Now Train Ensemble

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
 eclf = VotingClassifier(estimators=[rf_knn,ridge_knn,rf_cnn,rf_auto],voting = soft, njobs = -1)

for clf in [rf_knn,ridge_knn,rf_cnn,rf_auto,eclf]:
    scores_temp = 